<a href="https://colab.research.google.com/github/SallyPeter/gomycodeDSbootcamp/blob/main/ML/Support_vector_machines_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

What You're Aiming For

In this checkpoint, we are going to work on the 'Electric Vehicle Data' dataset that was provided by Kaggle as part of the Electric Vehicle Price Prediction competition.

Dataset description: This dataset contains information on the Battery Electric Vehicles (BEVs) and Plug-in Hybrid Electric Vehicles (PHEVs) that are currently registered with the Washington State Department of Licensing (DOL). This dataset was introduced as part of an official invitation-based competition on Kaggle. Our SVM model should answer the question "This is my car's model & make, along with a few other parameters, what price can this vehicle be brought or sold?”

➡️ Dataset link

https://i.imgur.com/IpuCW3s.jpg

➡️Columns explanation


**Instructions**

- Import you data and perform basic data exploration phase
  - Display general information about the dataset
  - Create a pandas profiling reports to gain insights into the dataset
  - Handle Missing and corrupted values
  - Remove duplicates, if they exist
  - Handle outliers, if they exist
  - Encode categorical features
- Select your target variable and the features
- Split your dataset to training and test sets
- Build and train an SVM model on the training set
- Assess your model performance on the test set using relevant evaluation metrics
- Discuss with your cohort alternative ways to improve your model performance

In [ ]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
# Import you data and perform basic data exploration phase
data = pd.read_csv("Electric_cars_dataset.csv")
data.head()

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k)
0,EV33174,5YJ3E1EC6L,Snohomish,LYNNWOOD,WA,98037.0,2020.0,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,0,32.0,109821694,POINT (-122.287614 47.83874),PUGET SOUND ENERGY INC,50
1,EV40247,JN1AZ0CP8B,Skagit,BELLINGHAM,WA,98229.0,2011.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,73,0,40.0,137375528,POINT (-122.414936 48.709388),PUGET SOUND ENERGY INC,15
2,EV12248,WBY1Z2C56F,Pierce,TACOMA,WA,98422.0,2015.0,BMW,I3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,81,0,27.0,150627382,POINT (-122.396286 47.293138),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,18
3,EV55713,1G1RD6E44D,King,REDMOND,WA,98053.0,2013.0,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,45.0,258766301,POINT (-122.024951 47.670286),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),33.9
4,EV28799,1G1FY6S05K,Pierce,PUYALLUP,WA,98375.0,2019.0,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238,0,25.0,296998138,POINT (-122.321062 47.103797),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,41.78


In [ ]:
data.shape

(64353, 18)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64353 entries, 0 to 64352
Data columns (total 18 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ID                                                 64353 non-null  object 
 1   VIN (1-10)                                         64353 non-null  object 
 2   County                                             64349 non-null  object 
 3   City                                               64344 non-null  object 
 4   State                                              64342 non-null  object 
 5   ZIP Code                                           64347 non-null  float64
 6   Model Year                                         64346 non-null  float64
 7   Make                                               64349 non-null  object 
 8   Model                                              64340 non-null  object 
 9   Electr

## Data Cleaning Part

First, we notice the Expected Price column is an object whereas it should be numeric

In [ ]:
data['Expected Price ($1k)'].value_counts().sort_index()

,count
Expected Price ($1k),
0,2
10,19
10.042,169
102,445
102.4,8
...,...
90,611
95,77
96,105


This is caused by some missing data with values as 'N/'.

This would be replaced by the mean value of cars in the same vehicle type and Clean Alternative Fuel Vehicle eligibility

In [ ]:
data[data['Expected Price ($1k)'] == 'N/']

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k)
2714,EV35011,1FMCU0LZ4N,King,YARROW POINT,WA,98004.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,48.0,192602371,POINT (-122.203169 47.619011),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),N/
4081,EV55301,1FMCU0KZ4N,Island,OAK HARBOR,WA,98277.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,10.0,193046381,POINT (-122.63841 48.310313),PUGET SOUND ENERGY INC,N/
13866,EV20774,1FMCU0LZ2N,Jefferson,PORT LUDLOW,WA,98365.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,24.0,190318901,POINT (-122.69441 47.905058),BONNEVILLE POWER ADMINISTRATION||PUGET SOUND E...,N/
14787,EV21204,1FMCU0KZ2N,Clark,CAMAS,WA,98607.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,18.0,195263764,POINT (-122.408498 45.620943),BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF C...,N/
16164,EV68630,1FMCU0KZ9N,King,BELLEVUE,WA,98006.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,41.0,187155314,POINT (-122.151342 47.560192),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),N/
22827,EV51468,1FMCU0KZ3N,King,SEATTLE,WA,98103.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,36.0,192317646,POINT (-122.343577 47.672491),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),N/
26799,EV42602,1FMCU0EZ0N,Pierce,GIG HARBOR,WA,98332.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,26.0,195845597,POINT (-122.599454 47.36582),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,N/
28511,EV51868,1FMCU0KZ1N,Clark,CAMAS,WA,98607.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,18.0,192539574,POINT (-122.408498 45.620943),BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF C...,N/
31081,EV41495,1FMCU0EZ4N,Pierce,TACOMA,WA,98444.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,29.0,192962717,POINT (-122.447791 47.1543),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,N/
37957,EV35977,1FMCU0LZ5N,Clark,VANCOUVER,WA,98662.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,49.0,193422764,POINT (-122.576308 45.677838),BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF C...,N/


In [ ]:
ford = data.loc[(data['Electric Vehicle Type'] == 'Plug-in Hybrid Electric Vehicle (PHEV)') &
 (data['Clean Alternative Fuel Vehicle (CAFV) Eligibility'] == 'Clean Alternative Fuel Vehicle Eligible') & (data['Make'] == 'FORD') & (data['Expected Price ($1k)'] != 'N/')]['Expected Price ($1k)'].astype(int)

ford.mean()

26.0

In [ ]:
data['Expected Price ($1k)'] = data['Expected Price ($1k)'].replace('N/', str(ford.mean()))

data['Expected Price ($1k)'].value_counts().sort_index()

,count
Expected Price ($1k),
0,2
10,19
10.042,169
102,445
102.4,8
...,...
9.079,51
90,611
95,77


Next, we convert the entire column to a float

In [ ]:
data['Expected Price ($1k)'] = data['Expected Price ($1k)'].astype(float)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64353 entries, 0 to 64352
Data columns (total 18 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ID                                                 64353 non-null  object 
 1   VIN (1-10)                                         64353 non-null  object 
 2   County                                             64349 non-null  object 
 3   City                                               64344 non-null  object 
 4   State                                              64342 non-null  object 
 5   ZIP Code                                           64347 non-null  float64
 6   Model Year                                         64346 non-null  float64
 7   Make                                               64349 non-null  object 
 8   Model                                              64340 non-null  object 
 9   Electr

In [ ]:
cat_col = data.select_dtypes(include='object')
cat_col.describe()

,ID,VIN (1-10),County,City,State,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Vehicle Location,Electric Utility
count,64353,64353,64349,64344,64342,64349,64340,64353,64353,63843,63631
unique,64353,5644,139,544,38,34,107,2,3,668,68
top,EV33174,5YJYGDEE9M,King,SEATTLE,WA,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,POINT (-122.122018 47.678465),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA)
freq,1,340,33552,11887,64168,27903,13138,47869,39948,1712,22555


In [ ]:
num_col = data.select_dtypes(include='number')
num_col.describe()

,ZIP Code,Model Year,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Expected Price ($1k)
count,64347.000000,64346.000000,64353.000000,64353.000000,64184.000000,6.435300e+04,64353.000000
mean,98143.452888,2018.186212,106.948985,2524.990754,29.951904,1.972905e+08,45.444041
std,2856.064329,2.726742,104.093919,12402.895104,14.661124,1.069466e+08,24.727252
min,745.000000,1993.000000,0.000000,0.000000,0.000000,4.385000e+03,0.000000
25%,98052.000000,2017.000000,14.000000,0.000000,19.000000,1.372865e+08,22.857000
50%,98121.000000,2018.000000,73.000000,0.000000,34.000000,1.753776e+08,40.000000
75%,98370.000000,2021.000000,215.000000,0.000000,43.000000,2.299039e+08,65.900000
max,99701.000000,2022.000000,337.000000,845000.000000,49.000000,4.789346e+08,1100.000000


In [ ]:
data.isnull().sum()

,0
ID,0
VIN (1-10),0
County,4
City,9
State,11
ZIP Code,6
Model Year,7
Make,4
Model,13
Electric Vehicle Type,0


Next, the location cordinate is split into logitude and latitude

In [ ]:
# r, cord = data['Vehicle Location'][1].replace("POINT", " ").strip(' ()').split(' ')


data['Longitude'] = data['Vehicle Location'].str.replace("POINT", " ").str.strip(' ()').str.split(' ', expand=True)[0]
data['Latitude'] = data['Vehicle Location'].str.replace("POINT", " ").str.strip(' ()').str.split(' ', expand=True)[1]

data.head()

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k),Longitude,Latitude
0,EV33174,5YJ3E1EC6L,Snohomish,LYNNWOOD,WA,98037.0,2020.0,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,0,32.0,109821694,POINT (-122.287614 47.83874),PUGET SOUND ENERGY INC,50.00,-122.287614,47.83874
1,EV40247,JN1AZ0CP8B,Skagit,BELLINGHAM,WA,98229.0,2011.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,73,0,40.0,137375528,POINT (-122.414936 48.709388),PUGET SOUND ENERGY INC,15.00,-122.414936,48.709388
2,EV12248,WBY1Z2C56F,Pierce,TACOMA,WA,98422.0,2015.0,BMW,I3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,81,0,27.0,150627382,POINT (-122.396286 47.293138),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,18.00,-122.396286,47.293138
3,EV55713,1G1RD6E44D,King,REDMOND,WA,98053.0,2013.0,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,45.0,258766301,POINT (-122.024951 47.670286),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),33.90,-122.024951,47.670286
4,EV28799,1G1FY6S05K,Pierce,PUYALLUP,WA,98375.0,2019.0,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238,0,25.0,296998138,POINT (-122.321062 47.103797),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,41.78,-122.321062,47.103797


Next, we look into filling the missing values in the State column

In [ ]:
data[data.State.isna()]

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k),Longitude,Latitude
820,EV235,1N4AZ0CP8E,Snohomish,EDMONDS,NaN,98020.0,2014.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,84,0,32.0,109001532,POINT (-122.370709 47.801416),PUGET SOUND ENERGY INC,19.000,-122.370709,47.801416
2520,EV237,1N4AZ1CP6J,Anchorage,CHUGIAK,NaN,99567.0,2018.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,151,0,NaN,311948836,POINT (-149.425108 61.41463),NaN,21.000,-149.425108,61.41463
6358,EV232,KNDJP3AE3J,Thurston,LACEY,NaN,98503.0,2018.0,KIA,SOUL EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,111,33950,2.0,135658435,POINT (-122.80237 47.02499),PUGET SOUND ENERGY INC,16.251,-122.80237,47.02499
35043,EV229,1N4AZ1CP1L,Snohomish,BOTHELL,NaN,98012.0,2020.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,149,0,1.0,141325628,POINT (-122.201515 47.843376),PUGET SOUND ENERGY INC,40.000,-122.201515,47.843376
35276,EV238,1N4AZ0CP8D,King,SEATTLE,NaN,98107.0,2013.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,75,0,43.0,212078681,NaN,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),18.000,NaN,NaN
35944,EV230,5YJYGDEFXL,Kitsap,BAINBRIDGE ISLAND,NaN,98110.0,2020.0,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291,0,23.0,107236958,POINT (-122.534497 47.643688),PUGET SOUND ENERGY INC,72.000,-122.534497,47.643688
41173,EV231,5YJSA1E20G,Yakima,YAKIMA,NaN,98901.0,2016.0,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,210,0,15.0,196516400,POINT (-120.441155 46.626077),NaN,65.000,-120.441155,46.626077
45990,EV228,1N4AZ0CP2D,King,VASHON,NaN,98070.0,2013.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,75,0,34.0,252897970,POINT (-122.469161 47.415443),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),18.000,-122.469161,47.415443
52823,EV234,KNDCC3LGXL,Kitsap,BREMERTON,NaN,98310.0,2020.0,KIA,NIRO,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,239,0,23.0,152551234,POINT (-122.623208 47.590898),PUGET SOUND ENERGY INC,33.000,-122.623208,47.590898
54532,EV233,3FA6P0SU2G,Thurston,LACEY,NaN,98516.0,2016.0,FORD,FUSION,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,19,0,22.0,111939852,POINT (-122.781876 47.089772),PUGET SOUND ENERGY INC,20.000,-122.781876,47.089772


In [ ]:
data[data['ZIP Code'] == 98020.0]['State'].unique()

array(['WA', nan], dtype=object)

We do this by creating a dictionary of zipcodes, where the values are the states the zipcodes belong to.

In [ ]:
# data.State.fillna(" ", inplace=True)
# data['ZIP Code'] = data['ZIP Code'].astype(str)

zip_state = {}

for each in np.array(data[['ZIP Code', 'State']]):
  if (not np.isnan(each[0]))  & (each[1] in ['WA', 'NC', 'CA','PA', 'IL', 'AE', 'NM', 'NY', 'MD', 'AP',  'VA', 'HI', 'CT', 'GA', 'TN', 'TX', 'DC', 'MS', 'RI', 'NE', 'NV', 'IN', 'WY', 'NJ', 'DE', 'KS', 'AK', 'OR', 'FL', 'PR', 'MA', 'MO', 'SD', 'MT', 'UT', 'LA', 'WI', 'AZ']) :
    # zip_state.add(tuple(each))
    zip_state[each[0]] = each[1]

The missing values are filled by mapping the zipcodes to the states

In [ ]:
data['State'] = data['State'].fillna(data['ZIP Code'].map(zip_state))

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64353 entries, 0 to 64352
Data columns (total 20 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ID                                                 64353 non-null  object 
 1   VIN (1-10)                                         64353 non-null  object 
 2   County                                             64349 non-null  object 
 3   City                                               64344 non-null  object 
 4   State                                              64352 non-null  object 
 5   ZIP Code                                           64347 non-null  float64
 6   Model Year                                         64346 non-null  float64
 7   Make                                               64349 non-null  object 
 8   Model                                              64340 non-null  object 
 9   Electr

This resolves all of the missing values in the State column except for one where the zip code occured only once. An option to resolve this could be using the long and lat to calculate the distance and assign this record to the state group nearest to it. As this is only one record, this might not be exactly necessary.

In [ ]:
data[data['State'].isna()]

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k),Longitude,Latitude
2520,EV237,1N4AZ1CP6J,Anchorage,CHUGIAK,NaN,99567.0,2018.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,151,0,NaN,311948836,POINT (-149.425108 61.41463),NaN,21.0,-149.425108,61.41463


In [ ]:
data[data['ZIP Code'] == 99567.0]

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k),Longitude,Latitude
2520,EV237,1N4AZ1CP6J,Anchorage,CHUGIAK,NaN,99567.0,2018.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,151,0,NaN,311948836,POINT (-149.425108 61.41463),NaN,21.0,-149.425108,61.41463


In [ ]:
data.isna().sum()

,0
ID,0
VIN (1-10),0
County,4
City,9
State,1
ZIP Code,6
Model Year,7
Make,4
Model,13
Electric Vehicle Type,0


Following this, we fill na for city using the zip code as seen earlier

In [ ]:
zip_city = {}

for each in np.array(data[['ZIP Code', 'City']]):
  if (not np.isnan(each[0]))  :
    # zip_state.add(tuple(each))
    zip_city[each[0]] = each[1]



data['City'] = data['City'].fillna(data['ZIP Code'].map(zip_city))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64353 entries, 0 to 64352
Data columns (total 20 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ID                                                 64353 non-null  object 
 1   VIN (1-10)                                         64353 non-null  object 
 2   County                                             64349 non-null  object 
 3   City                                               64353 non-null  object 
 4   State                                              64352 non-null  object 
 5   ZIP Code                                           64347 non-null  float64
 6   Model Year                                         64346 non-null  float64
 7   Make                                               64349 non-null  object 
 8   Model                                              64340 non-null  object 
 9   Electr

Same goes for the County

In [ ]:
zip_county = {}

for each in np.array(data[['ZIP Code', 'County']]):
  if (not np.isnan(each[0]))  :
    # zip_state.add(tuple(each))
    zip_county[each[0]] = each[1]



data['County'] = data['County'].fillna(data['ZIP Code'].map(zip_county))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64353 entries, 0 to 64352
Data columns (total 20 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ID                                                 64353 non-null  object 
 1   VIN (1-10)                                         64353 non-null  object 
 2   County                                             64349 non-null  object 
 3   City                                               64353 non-null  object 
 4   State                                              64352 non-null  object 
 5   ZIP Code                                           64347 non-null  float64
 6   Model Year                                         64346 non-null  float64
 7   Make                                               64349 non-null  object 
 8   Model                                              64340 non-null  object 
 9   Electr

We notice some records in County appeared only once in the dataset, the zipcode and the State is also uncommon appearing only once in the dataet.

This records would later be removed as other columns which may be useful for our analysis are missing

In [ ]:
data[data['County'].isna()]

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k),Longitude,Latitude
2195,EV45994,5YJ3E1EB1J,NaN,APO,AE,9751.0,2018.0,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,NaN,475911509,NaN,NaN,69.0,NaN,NaN
5913,EV39990,1C4JJXR67M,NaN,FPO,AP,96349.0,2021.0,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,21,0,NaN,187228030,NaN,NaN,40.0,NaN,NaN
40262,EV7503,1G1RA6E49F,NaN,FPO,AP,96678.0,2015.0,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,NaN,232819723,NaN,NaN,18.5,NaN,NaN
52218,EV69,1FADP5FU6H,NaN,FPO,AP,96662.0,2017.0,FORD,C-MAX,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,20,0,NaN,138149357,NaN,NaN,21.0,NaN,NaN


We alo try to fill the legislative district using the same technique

In [ ]:
zip_ld = {}

for each in np.array(data[['ZIP Code', 'Legislative District']]):
  if (not np.isnan(each[0]))  & (not np.isnan(each[1]))  :
    # zip_state.add(tuple(each))
    zip_ld[each[0]] = each[1]



data['Legislative District'] = data['Legislative District'].fillna(data['ZIP Code'].map(zip_ld))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64353 entries, 0 to 64352
Data columns (total 20 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ID                                                 64353 non-null  object 
 1   VIN (1-10)                                         64353 non-null  object 
 2   County                                             64349 non-null  object 
 3   City                                               64353 non-null  object 
 4   State                                              64352 non-null  object 
 5   ZIP Code                                           64347 non-null  float64
 6   Model Year                                         64346 non-null  float64
 7   Make                                               64349 non-null  object 
 8   Model                                              64340 non-null  object 
 9   Electr

In [ ]:
zip_ld = {}

for each in np.array(data[['ZIP Code', 'Legislative District']]):
  if (not np.isnan(each[0]))  & (not np.isnan(each[1]))  :
    # zip_state.add(tuple(each))
    zip_ld[each[0]] = each[1]



data['Legislative District'] = data['Legislative District'].fillna(data['ZIP Code'].map(zip_ld))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64353 entries, 0 to 64352
Data columns (total 20 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ID                                                 64353 non-null  object 
 1   VIN (1-10)                                         64353 non-null  object 
 2   County                                             64349 non-null  object 
 3   City                                               64353 non-null  object 
 4   State                                              64352 non-null  object 
 5   ZIP Code                                           64347 non-null  float64
 6   Model Year                                         64346 non-null  float64
 7   Make                                               64349 non-null  object 
 8   Model                                              64340 non-null  object 
 9   Electr

This does not really fill in the missing data as expected.

In [ ]:
data[data['Legislative District'].isna()]

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k),Longitude,Latitude
85,EV91250,1FTFW1ED3M,Cumberland,FAYETTEVILLE,NC,28311.0,2021.0,FORD,F-150,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,NaN,148274808,POINT (-78.891986 35.143971),NaN,43.00,-78.891986,35.143971
787,EV3014,KNDCD3LD6N,Marin,SAN RAFAEL,CA,94901.0,2022.0,KIA,NIRO,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,26,0,NaN,181521700,POINT (-122.51209 37.973771),NaN,24.69,-122.51209,37.973771
1465,EV68073,3FMTK3SU9M,Montgomery,FORT WASHINGTON,PA,19034.0,2021.0,FORD,MACH-E,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,NaN,172153180,POINT (-75.204534 40.135618),NaN,42.80,-75.204534,40.135618
1830,EV18703,5YJ3E1EB3J,Lake,HIGHLAND PARK,IL,60035.0,2018.0,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,NaN,128455015,POINT (-87.808789 42.182578),NaN,69.00,-87.808789,42.182578
2195,EV45994,5YJ3E1EB1J,NaN,APO,AE,9751.0,2018.0,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,NaN,475911509,NaN,NaN,69.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61614,EV33766,1G1RC6E48B,Prince William,WOODBRIDGE,VA,22191.0,2011.0,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,35,0,NaN,144232399,POINT (-77.272331 38.627945),NaN,7.00,-77.272331,38.627945
63329,EV16543,5YJ3E1EB0J,Alexandria City,ALEXANDRIA,VA,22301.0,2018.0,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,NaN,261314294,POINT (-77.059798 38.819698),NaN,69.00,-77.059798,38.819698
63772,EV74159,5YJ3E1EBXK,Frederick,FREDERICK,MD,21704.0,2019.0,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,220,0,NaN,114252337,POINT (-77.368491 39.354463),NaN,57.00,-77.368491,39.354463
64151,EV43314,2C4RC1N72J,Anne Arundel,PASADENA,MD,21122.0,2018.0,CHRYSLER,PACIFICA,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,33,0,NaN,269209055,POINT (-76.514013 39.124897),NaN,30.00,-76.514013,39.124897


We also use same format to fill missing values in the Electric Utility column

In [ ]:
# data[data['Electric Utility'].isna()]

zip_eu = {}

for each in np.array(data[['ZIP Code', 'Electric Utility']]):
  if (not np.isnan(each[0])) & (not pd.isna(each[1]))  :
    # zip_state.add(tuple(each))
    zip_eu[each[0]] = each[1]

data['Electric Utility'] = data['Electric Utility'].fillna(data['ZIP Code'].map(zip_eu))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64353 entries, 0 to 64352
Data columns (total 20 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ID                                                 64353 non-null  object 
 1   VIN (1-10)                                         64353 non-null  object 
 2   County                                             64349 non-null  object 
 3   City                                               64353 non-null  object 
 4   State                                              64352 non-null  object 
 5   ZIP Code                                           64347 non-null  float64
 6   Model Year                                         64346 non-null  float64
 7   Make                                               64349 non-null  object 
 8   Model                                              64340 non-null  object 
 9   Electr

In [ ]:
data.isna().sum()

,0
ID,0
VIN (1-10),0
County,4
City,0
State,1
ZIP Code,6
Model Year,7
Make,4
Model,13
Electric Vehicle Type,0


In [ ]:
data[data.Model.isna()]

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k),Longitude,Latitude
2714,EV35011,1FMCU0LZ4N,King,YARROW POINT,WA,98004.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,48.0,192602371,POINT (-122.203169 47.619011),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),26.0,-122.203169,47.619011
4081,EV55301,1FMCU0KZ4N,Island,OAK HARBOR,WA,98277.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,10.0,193046381,POINT (-122.63841 48.310313),PUGET SOUND ENERGY INC,26.0,-122.63841,48.310313
13866,EV20774,1FMCU0LZ2N,Jefferson,PORT LUDLOW,WA,98365.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,24.0,190318901,POINT (-122.69441 47.905058),BONNEVILLE POWER ADMINISTRATION||PUGET SOUND E...,26.0,-122.69441,47.905058
14787,EV21204,1FMCU0KZ2N,Clark,CAMAS,WA,98607.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,18.0,195263764,POINT (-122.408498 45.620943),BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF C...,26.0,-122.408498,45.620943
16164,EV68630,1FMCU0KZ9N,King,BELLEVUE,WA,98006.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,41.0,187155314,POINT (-122.151342 47.560192),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),26.0,-122.151342,47.560192
22827,EV51468,1FMCU0KZ3N,King,SEATTLE,WA,98103.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,36.0,192317646,POINT (-122.343577 47.672491),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),26.0,-122.343577,47.672491
26799,EV42602,1FMCU0EZ0N,Pierce,GIG HARBOR,WA,98332.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,26.0,195845597,POINT (-122.599454 47.36582),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,26.0,-122.599454,47.36582
28511,EV51868,1FMCU0KZ1N,Clark,CAMAS,WA,98607.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,18.0,192539574,POINT (-122.408498 45.620943),BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF C...,26.0,-122.408498,45.620943
31081,EV41495,1FMCU0EZ4N,Pierce,TACOMA,WA,98444.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,29.0,192962717,POINT (-122.447791 47.1543),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,26.0,-122.447791,47.1543
37957,EV35977,1FMCU0LZ5N,Clark,VANCOUVER,WA,98662.0,2022.0,FORD,NaN,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,49.0,193422764,POINT (-122.576308 45.677838),BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF C...,26.0,-122.576308,45.677838


Next, we fill the missing values in the Model column using the mapping to the zip codes

In [ ]:
# cdata = data.copy()

ford_zip = list(data[data.Model.isna()]['ZIP Code'])

# for each in ford_zip:
#   data['Model'] =

for zip_code in ford_zip:
    mode_model = data[(data['ZIP Code'] == zip_code) & (data['Make'] == 'FORD')]['Model'].mode()
    if not mode_model.empty:
        data.loc[(data['ZIP Code'] == zip_code) & (data['Make'] == 'FORD') & (data['Model'].isna()), 'Model'] = mode_model[0]



# data[(data['ZIP Code']==98004.0) & (data['Make']=='FORD')]['Model'].mode()

data[data.Model.isna()]

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k),Longitude,Latitude


In [ ]:
data.isna().sum()

,0
ID,0
VIN (1-10),0
County,4
City,0
State,1
ZIP Code,6
Model Year,7
Make,4
Model,0
Electric Vehicle Type,0


Then we drop the records with missing values in the County Column

In [ ]:
data.drop(data[data.County.isna()].index, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64349 entries, 0 to 64352
Data columns (total 20 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ID                                                 64349 non-null  object 
 1   VIN (1-10)                                         64349 non-null  object 
 2   County                                             64349 non-null  object 
 3   City                                               64349 non-null  object 
 4   State                                              64348 non-null  object 
 5   ZIP Code                                           64343 non-null  float64
 6   Model Year                                         64342 non-null  float64
 7   Make                                               64345 non-null  object 
 8   Model                                              64349 non-null  object 
 9   Electric Ve

We also drop records without a vehicle location

In [ ]:
data = data.drop(data[data['Vehicle Location'].isna()].index)
# data.info()
# data.isna().sum()

In [ ]:
data.isna().sum()

,0
ID,0
VIN (1-10),0
County,0
City,0
State,1
ZIP Code,4
Model Year,7
Make,4
Model,0
Electric Vehicle Type,0


We drop the records with missing Zip Codes

In [ ]:
# data.drop(data['Electric Utility'].isna().index, inplace=True)
data = data.drop(data[data['ZIP Code'].isna()].index)
data.isna().sum()

,0
ID,0
VIN (1-10),0
County,0
City,0
State,1
ZIP Code,0
Model Year,7
Make,4
Model,0
Electric Vehicle Type,0


We fill the Make column using the mapping technique between Make and Model

In [ ]:
# data[data['Model'] == 'LEAF']['Make'].mode()


make_model = {}

for each in np.array(data[['Model', 'Make']]):
  if each[0] :
    if each[1]:
    # zip_state.add(tuple(each))
      make_model[each[0]] = each[1]


data['Make'] = data['Make'].fillna(data['Model'].map(make_model))
data.isna().sum()

,0
ID,0
VIN (1-10),0
County,0
City,0
State,1
ZIP Code,0
Model Year,7
Make,0
Model,0
Electric Vehicle Type,0


Next, we fill the missing values in the Model Year column

In [ ]:
# cdata[(cdata['ZIP Code'] == 98004.0) & (cdata['Make'] == 'FORD')]['Model'].mode()

data[data['Model Year'].isna()]

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k),Longitude,Latitude
18618,EV258,KMHE34L17G,Spokane,SPOKANE VALLEY,WA,99016.0,NaN,HYUNDAI,SONATA PLUG-IN HYBRID,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,27,34600,4.0,476776030,POINT (-117.152872 47.645579),BONNEVILLE POWER ADMINISTRATION||AVISTA CORP||...,16.365,-117.152872,47.645579
34663,EV255,5YJ3E1EB6K,King,SEATTLE,WA,98115.0,NaN,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,220,0,46.0,192220731,POINT (-122.297534 47.685291),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),57.000,-122.297534,47.685291
35578,EV250,5YJSA1CP9D,Clark,VANCOUVER,WA,98683.0,NaN,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,208,69900,17.0,193870848,POINT (-122.510748 45.603727),BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF C...,33.890,-122.510748,45.603727
50803,EV252,5YJYGDEF0L,Chelan,WENATCHEE,WA,98801.0,NaN,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291,0,12.0,127404371,POINT (-120.336499 47.425146),PUD NO 1 OF CHELAN COUNTY,72.000,-120.336499,47.425146
54860,EV249,WAUTPBFF2G,Snohomish,EDMONDS,WA,98026.0,NaN,AUDI,A3,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,16,0,21.0,165842008,POINT (-122.333046 47.829439),PUGET SOUND ENERGY INC,18.860,-122.333046,47.829439
59409,EV256,5YJ3E1EB0M,Clark,CAMAS,WA,98607.0,NaN,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,18.0,169660508,POINT (-122.408498 45.620943),BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF C...,64.000,-122.408498,45.620943
62552,EV253,5YJ3E1EA2M,King,SEATTLE,WA,98144.0,NaN,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,37.0,150168735,POINT (-122.30033 47.585339),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),64.000,-122.30033,47.585339


In [ ]:
data[(data['Make']== 'HYUNDAI') & (data['Model'] == 'SONATA PLUG-IN HYBRID') & (data['City'] == 'SPOKANE VALLEY')]['Model Year'].mode()

,Model Year
0,2016.0


In [ ]:
zip_mmy = {}

for each in np.array(data[['Make', 'Model', 'City', 'Model Year']]):
  key = str(each[0]) + str(each[1]) + str(each[2])
  zip_mmy[key] = each[3]


data['key'] = data['Make'].astype(str) + data['Model'].astype(str) + data['City'].astype(str)
data['Model Year'] = data['Model Year'].fillna(data['key'].map(zip_mmy))

data['Model Year'] = data['Model Year'].fillna(data['key'].mode())
data.isna().sum()

<ipython-input-39-938e9f888e64>:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Model Year'] = data['Model Year'].fillna(data['key'].mode())


,0
ID,0
VIN (1-10),0
County,0
City,0
State,1
ZIP Code,0
Model Year,1
Make,0
Model,0
Electric Vehicle Type,0


Lastly we drop the records with missing values in the legislative District column

In [ ]:
data = data.drop(data[data['Legislative District'].isna()].index)
data.isna().sum()

,0
ID,0
VIN (1-10),0
County,0
City,0
State,0
ZIP Code,0
Model Year,1
Make,0
Model,0
Electric Vehicle Type,0


In [ ]:
make = list(data[data['Electric Utility'].isna()]['Make'])
model = list(data[data['Electric Utility'].isna()]['Model'])

make_model = [f"{m} {mo}" for m, mo in zip(make, model)]

# for each in ford_zip:
#   data['Model'] =

for each in make_model:
    mode_model = data[(data['Make'] == each.split(" ", 1)[0]) & (data['Model'] ==each.split(" ", 1)[1])]['Electric Utility'].mode()
    if not mode_model.empty:
        data.loc[(data['Make'] == each.split(" ", 1)[0]) & (data['Model'] == each.split(" ", 1)[1]) & (data['Electric Utility'].isna()), 'Electric Utility'] = mode_model[0]



# data[(data['ZIP Code']==98004.0) & (data['Make']=='FORD')]['Model'].mode()

data[data['Electric Utility'].isna()]

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,...,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k),Longitude,Latitude,key


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63674 entries, 0 to 64352
Data columns (total 21 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ID                                                 63674 non-null  object 
 1   VIN (1-10)                                         63674 non-null  object 
 2   County                                             63674 non-null  object 
 3   City                                               63674 non-null  object 
 4   State                                              63674 non-null  object 
 5   ZIP Code                                           63674 non-null  float64
 6   Model Year                                         63673 non-null  float64
 7   Make                                               63674 non-null  object 
 8   Model                                              63674 non-null  object 
 9   Electric Ve

In [ ]:
data.drop('key', axis = 1, inplace=True)

In [ ]:
data.head(2).T

,0,1
ID,EV33174,EV40247
VIN (1-10),5YJ3E1EC6L,JN1AZ0CP8B
County,Snohomish,Skagit
City,LYNNWOOD,BELLINGHAM
State,WA,WA
ZIP Code,98037.0,98229.0
Model Year,2020.0,2011.0
Make,TESLA,NISSAN
Model,MODEL 3,LEAF
Electric Vehicle Type,Battery Electric Vehicle (BEV),Battery Electric Vehicle (BEV)


In [ ]:
data['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].unique()

array(['Clean Alternative Fuel Vehicle Eligible',
       'Not eligible due to low battery range',
       'Eligibility unknown as battery range has not been researched'],
      dtype=object)

In [ ]:
cafv = {'Clean Alternative Fuel Vehicle Eligible':0, 'Not eligible due to low battery range': 1, 'Eligibility unknown as battery range has not been researched': 2}
data['Clean Alternative Fuel Vehicle (CAFV) Eligibility'] = data['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].map(cafv)
data['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].unique()

array([0, 1, 2])

In [ ]:
data['Electric Vehicle Type'].unique()

array(['Battery Electric Vehicle (BEV)',
       'Plug-in Hybrid Electric Vehicle (PHEV)'], dtype=object)

In [ ]:
evt = {'Battery Electric Vehicle (BEV)':0, 'Plug-in Hybrid Electric Vehicle (PHEV)':1}

data['Electric Vehicle Type'] = data['Electric Vehicle Type'].map(evt)

data['Electric Vehicle Type'].unique()

array([0, 1])

In [ ]:
data['Electric Utility'].unique()

array(['PUGET SOUND ENERGY INC',
       'BONNEVILLE POWER ADMINISTRATION||CITY OF TACOMA - (WA)||PENINSULA LIGHT COMPANY',
       'PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA)',
       'BONNEVILLE POWER ADMINISTRATION||CITY OF TACOMA - (WA)||ELMHURST MUTUAL POWER & LIGHT CO|PENINSULA LIGHT COMPANY',
       'BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF CLARK COUNTY - (WA)',
       'MODERN ELECTRIC WATER COMPANY',
       'CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA)',
       'BONNEVILLE POWER ADMINISTRATION||CITY OF TACOMA - (WA)||PARKLAND LIGHT & WATER COMPANY|PENINSULA LIGHT COMPANY',
       'BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF KLICKITAT COUNTY',
       'BONNEVILLE POWER ADMINISTRATION||PUD 1 OF SNOHOMISH COUNTY',
       'PUD NO 1 OF DOUGLAS COUNTY',
       'BONNEVILLE POWER ADMINISTRATION||PUGET SOUND ENERGY INC||PUD NO 1 OF JEFFERSON COUNTY',
       'PUGET SOUND ENERGY INC||PUD NO 1 OF WHATCOM COUNTY',
       'BONNEVILLE POWER ADMINISTRATION||CITY OF TACOMA - (WA)||LAKEVIEW

Replace the Electric Utility with the number of options each record has. This is seperated by || characters and the count represents the number of options each record has

In [ ]:
data['Electric Utility'] = data['Electric Utility'].str.split("\|\|").apply(len)
data.head(2)

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k),Longitude,Latitude
0,EV33174,5YJ3E1EC6L,Snohomish,LYNNWOOD,WA,98037.0,2020.0,TESLA,MODEL 3,0,0,308,0,32.0,109821694,POINT (-122.287614 47.83874),1,50.0,-122.287614,47.83874
1,EV40247,JN1AZ0CP8B,Skagit,BELLINGHAM,WA,98229.0,2011.0,NISSAN,LEAF,0,0,73,0,40.0,137375528,POINT (-122.414936 48.709388),1,15.0,-122.414936,48.709388


In [ ]:
c_count = dict(data.City.value_counts())

cities = {}

# Iterate through the city_data dictionary
for city, value in c_count.items():
  if value >=500:
    range_key = (value // 500) * 500
  else:
    range_key = 'Below 500'

    # Add the city to the corresponding range
  if range_key not in cities:
        cities[range_key] = []

  cities[range_key].append(city)

In [ ]:
# cities.keys()
def map_city(city):
    for k, v in cities.items():
        if city in v:
            return city_key[k]
    return city_key['Below 500']


city_key = {11000: 0, 3000: 1, 2000: 2, 1500: 3, 1000:4, 500:5, 'Below 500':6}

data['City'] = data['City'].apply(map_city)

data.head()

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k),Longitude,Latitude
0,EV33174,5YJ3E1EC6L,Snohomish,5,WA,98037.0,2020.0,TESLA,MODEL 3,0,0,308,0,32.0,109821694,POINT (-122.287614 47.83874),1,50.00,-122.287614,47.83874
1,EV40247,JN1AZ0CP8B,Skagit,4,WA,98229.0,2011.0,NISSAN,LEAF,0,0,73,0,40.0,137375528,POINT (-122.414936 48.709388),1,15.00,-122.414936,48.709388
2,EV12248,WBY1Z2C56F,Pierce,4,WA,98422.0,2015.0,BMW,I3,0,0,81,0,27.0,150627382,POINT (-122.396286 47.293138),3,18.00,-122.396286,47.293138
3,EV55713,1G1RD6E44D,King,2,WA,98053.0,2013.0,CHEVROLET,VOLT,1,0,38,0,45.0,258766301,POINT (-122.024951 47.670286),2,33.90,-122.024951,47.670286
4,EV28799,1G1FY6S05K,Pierce,5,WA,98375.0,2019.0,CHEVROLET,BOLT EV,0,0,238,0,25.0,296998138,POINT (-122.321062 47.103797),3,41.78,-122.321062,47.103797


In [ ]:
data.drop(data[data['Model'] == '$16.36K'].index, inplace = True)
data[data['Model'] == '$16.36K']

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k),Longitude,Latitude


In [ ]:
le = LabelEncoder()


data['Make'] = le.fit_transform(data['Make'])
data['Model'] = le.fit_transform(data['Model'])


In [ ]:
data['Longitude'] = pd.to_numeric(data['Longitude'], errors='coerce')
data['Latitude'] = pd.to_numeric(data['Latitude'], errors='coerce')

data.head()

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k),Longitude,Latitude
0,EV33174,5YJ3E1EC6L,Snohomish,5,WA,98037.0,2020.0,28,56,0,0,308,0,32.0,109821694,POINT (-122.287614 47.83874),1,50.00,-122.287614,47.838740
1,EV40247,JN1AZ0CP8B,Skagit,4,WA,98229.0,2011.0,22,52,0,0,73,0,40.0,137375528,POINT (-122.414936 48.709388),1,15.00,-122.414936,48.709388
2,EV12248,WBY1Z2C56F,Pierce,4,WA,98422.0,2015.0,3,44,0,0,81,0,27.0,150627382,POINT (-122.396286 47.293138),3,18.00,-122.396286,47.293138
3,EV55713,1G1RD6E44D,King,2,WA,98053.0,2013.0,5,95,1,0,38,0,45.0,258766301,POINT (-122.024951 47.670286),2,33.90,-122.024951,47.670286
4,EV28799,1G1FY6S05K,Pierce,5,WA,98375.0,2019.0,5,14,0,0,238,0,25.0,296998138,POINT (-122.321062 47.103797),3,41.78,-122.321062,47.103797


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63673 entries, 0 to 64352
Data columns (total 20 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ID                                                 63673 non-null  object 
 1   VIN (1-10)                                         63673 non-null  object 
 2   County                                             63673 non-null  object 
 3   City                                               63673 non-null  int64  
 4   State                                              63673 non-null  object 
 5   ZIP Code                                           63673 non-null  float64
 6   Model Year                                         63672 non-null  float64
 7   Make                                               63673 non-null  int64  
 8   Model                                              63673 non-null  int64  
 9   Electric Ve

In [ ]:
data[data['Model Year'].isna()]

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k),Longitude,Latitude
54860,EV249,WAUTPBFF2G,Snohomish,5,WA,98026.0,NaN,0,7,1,1,16,0,21.0,165842008,POINT (-122.333046 47.829439),1,18.86,-122.333046,47.829439


In [ ]:
data.loc[(data['County'] =='Snohomish') & (data['City'] == 5) & (data['Make'] == 0) & (data['Model'] == 7)]['Model Year'].mode()

,Model Year
0,2017.0


In [ ]:
data['Model Year'].fillna(2017)

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63673 entries, 0 to 64352
Data columns (total 20 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ID                                                 63673 non-null  object 
 1   VIN (1-10)                                         63673 non-null  object 
 2   County                                             63673 non-null  object 
 3   City                                               63673 non-null  int64  
 4   State                                              63673 non-null  object 
 5   ZIP Code                                           63673 non-null  float64
 6   Model Year                                         63672 non-null  float64
 7   Make                                               63673 non-null  int64  
 8   Model                                              63673 non-null  int64  
 9   Electric Ve

In [ ]:
data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63673 entries, 0 to 64352
Data columns (total 20 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ID                                                 63673 non-null  object 
 1   VIN (1-10)                                         63673 non-null  object 
 2   County                                             63673 non-null  object 
 3   City                                               63673 non-null  int64  
 4   State                                              63673 non-null  object 
 5   ZIP Code                                           63673 non-null  float64
 6   Model Year                                         63672 non-null  float64
 7   Make                                               63673 non-null  int64  
 8   Model                                              63673 non-null  int64  
 9   Electric Ve

In [ ]:
data.drop(index=54860, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63672 entries, 0 to 64352
Data columns (total 20 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ID                                                 63672 non-null  object 
 1   VIN (1-10)                                         63672 non-null  object 
 2   County                                             63672 non-null  object 
 3   City                                               63672 non-null  int64  
 4   State                                              63672 non-null  object 
 5   ZIP Code                                           63672 non-null  float64
 6   Model Year                                         63672 non-null  float64
 7   Make                                               63672 non-null  int64  
 8   Model                                              63672 non-null  int64  
 9   Electric Ve

Finally, Clean data!!!

In [ ]:
data.columns

Index(['ID', 'VIN (1-10)', 'County', 'City', 'State', 'ZIP Code', 'Model Year',
       'Make', 'Model', 'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range',
       'Base MSRP', 'Legislative District', 'DOL Vehicle ID',
       'Vehicle Location', 'Electric Utility', 'Expected Price ($1k)',
       'Longitude', 'Latitude'],
      dtype='object')

In [ ]:
# Select your target variable and the features

x = data.drop(['ID', 'VIN (1-10)', 'County', 'State', 'ZIP Code', 'DOL Vehicle ID', 'Vehicle Location', 'Expected Price ($1k)'], axis=1)
y = data['Expected Price ($1k)']

In [ ]:
x.head()

,City,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,Electric Utility,Longitude,Latitude
0,5,2020.0,28,56,0,0,308,0,32.0,1,-122.287614,47.838740
1,4,2011.0,22,52,0,0,73,0,40.0,1,-122.414936,48.709388
2,4,2015.0,3,44,0,0,81,0,27.0,3,-122.396286,47.293138
3,2,2013.0,5,95,1,0,38,0,45.0,2,-122.024951,47.670286
4,5,2019.0,5,14,0,0,238,0,25.0,3,-122.321062,47.103797


In [ ]:
y.head()

,Expected Price ($1k)
0,50.00
1,15.00
2,18.00
3,33.90
4,41.78


In [ ]:
# Split your dataset to training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [ ]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44570 entries, 47095 to 22590
Data columns (total 12 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   City                                               44570 non-null  int64  
 1   Model Year                                         44570 non-null  float64
 2   Make                                               44570 non-null  int64  
 3   Model                                              44570 non-null  int64  
 4   Electric Vehicle Type                              44570 non-null  int64  
 5   Clean Alternative Fuel Vehicle (CAFV) Eligibility  44570 non-null  int64  
 6   Electric Range                                     44570 non-null  int64  
 7   Base MSRP                                          44570 non-null  int64  
 8   Legislative District                               44570 non-null  float64
 9   Electri

In [ ]:
# Build and train an SVM model on the training set
model = svm.SVR()

model.fit(x_train, y_train)
y_pred = model.predict(x_test)


In [ ]:
# Assess your model performance on the test set using relevant evaluation metrics
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print('r2: ', r2)
print('mae: ', mae)
print('mse: ', mse)

r2:  0.044134282834088046
mae:  19.802475798497976
mse:  626.3987432244543


In [ ]:
# Discuss with your cohort alternative ways to improve your model performance
model = svm.SVR(kernel="poly", C=100, gamma="auto", degree=3)

model.fit(x_train, y_train)
y_pred = model.predict(x_test)


In [ ]:
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print('r2: ', r2)
print('mae: ', mae)
print('mse: ', mse)

In [ ]:
model

SVR()

In [ ]:
# yr_2022 = data[data['Model Year'] == 2022.0]
# yr_2022.head()

In [ ]:
# yr_2022_ford = data.loc[(data['Electric Vehicle Type'] == 'Plug-in Hybrid Electric Vehicle (PHEV)') & (data['Clean Alternative Fuel Vehicle (CAFV) Eligibility'] == 'Clean Alternative Fuel Vehicle Eligible') &
#                              (data['Make'] == 'FORD')]

# yr_2022_ford.head()

In [ ]:
# yr_2022_ford = yr_2022_ford[yr_2022_ford['Expected Price ($1k)'] != 'N/']
# yr_2022_ford['Expected Price ($1k)'] = yr_2022_ford['Expected Price ($1k)'].astype(int)
# yr_2022_ford['Expected Price ($1k)'].mean()

In [ ]:
# yr_2022 = data[data['Model Year'] == 2022.0]
# yr_2022_ford = data.loc[(data['Electric Vehicle Type'] == 'Plug-in Hybrid Electric Vehicle (PHEV)') & (data['Clean Alternative Fuel Vehicle (CAFV) Eligibility'] == 'Clean Alternative Fuel Vehicle Eligible') &
#                              (data['Make'] == 'FORD')]
# yr_2022_ford = yr_2022_ford[yr_2022_ford['Expected Price ($1k)'] != 'N/']
# yr_2022_ford['Expected Price ($1k)'] = yr_2022_ford['Expected Price ($1k)'].astype(int)
# yr_2022_ford['Expected Price ($1k)'].mean()